<a href="https://colab.research.google.com/github/BHANAPEESHAN/CNN_Projects_/blob/main/TL_Models_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']="bhanapeeshan"
os.environ['KAGGLE_KEY']="646853511eaba7e71ae9a0d4d3c4af8a"
!kaggle datasets download -d gpiosenka/musical-instruments-image-classification

 91% 186M/204M [00:05<00:00, 34.8MB/s]
100% 204M/204M [00:05<00:00, 41.3MB/s]


In [2]:
from zipfile import ZipFile
file_name = "/content/musical-instruments-image-classification.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done Unzipping')

Done Unzipping


In [3]:
import warnings
warnings.filterwarnings('ignore')
data_dir_list=os.listdir("/content/train")
print(data_dir_list)
path,dirs,files=next(os.walk("/content/train"))
file_count=len(files)

['casaba', 'bagpipes', 'concertina', 'Tambourine', 'harp', 'trombone', 'Xylophone', 'sitar', 'tuba', 'clarinet', 'dulcimer', 'alphorn', 'clavichord', 'guitar', 'piano', 'saxaphone', 'ocarina', 'bongo drum', 'banjo', 'trumpet', 'Didgeridoo', 'harmonica', 'guiro', 'flute', 'castanets', 'steel drum', 'acordian', 'marakas', 'violin', 'drums']


In [4]:
len(data_dir_list)

30

In [5]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
batch_size=32

In [6]:
!nvidia-smi

Thu Mar  3 13:49:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:

# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
model_dictionary

{'DenseNet121': <function keras.applications.densenet.DenseNet121>,
 'DenseNet169': <function keras.applications.densenet.DenseNet169>,
 'DenseNet201': <function keras.applications.densenet.DenseNet201>,
 'EfficientNetB0': <function keras.applications.efficientnet.EfficientNetB0>,
 'EfficientNetB1': <function keras.applications.efficientnet.EfficientNetB1>,
 'EfficientNetB2': <function keras.applications.efficientnet.EfficientNetB2>,
 'EfficientNetB3': <function keras.applications.efficientnet.EfficientNetB3>,
 'EfficientNetB4': <function keras.applications.efficientnet.EfficientNetB4>,
 'EfficientNetB5': <function keras.applications.efficientnet.EfficientNetB5>,
 'EfficientNetB6': <function keras.applications.efficientnet.EfficientNetB6>,
 'EfficientNetB7': <function keras.applications.efficientnet.EfficientNetB7>,
 'EfficientNetV2B0': <function keras.applications.efficientnet_v2.EfficientNetV2B0>,
 'EfficientNetV2B1': <function keras.applications.efficientnet_v2.EfficientNetV2B1>,
 '

In [17]:
def preprocess_data(train, validation, batch_size, img_size):
    
    train_datagen=ImageDataGenerator(rescale=1/255.0,rotation_range=0.4,horizontal_flip=True,vertical_flip=True,shear_range=0.3,zoom_range=0.25)
    train_gen=train_datagen.flow_from_directory(train,
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            target_size=img_size)
   
    val_datagen=ImageDataGenerator(rescale=1/255.0)
    val_gen=val_datagen.flow_from_directory(validation,
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            target_size=img_size)
    return train_gen,val_gen

train_processed_224, validation_processed_224 = preprocess_data( "/content/train", "/content/valid", batch_size=batch_size, img_size=[224,224])
train_processed_331, validation_processed_331 = preprocess_data( "/content/train", "/content/valid", batch_size=batch_size, img_size=[331,331])


Found 4793 images belonging to 30 classes.
Found 150 images belonging to 30 classes.
Found 4793 images belonging to 30 classes.
Found 150 images belonging to 30 classes.


In [19]:
number=0
for i in data_dir_list:
  number=number+len(os.listdir("/content/train/"+i+"/"))
number

4793

In [20]:


num_train = number
num_iterations = int(num_train/batch_size)

# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_gen = train_processed_331
        val_gen = validation_processed_331
    else:
        input_shape=(224,224,3)
        train_gen = train_processed_224
        val_gen= validation_processed_224
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    pre_trained_model = model(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False
    
    # custom modifications on top of pre-trained model and fit
    clf_model = tf.keras.models.Sequential()
    clf_model.add(pre_trained_model)
    clf_model.add(tf.keras.layers.Dense(len(data_dir_list), activation='softmax'))
    clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    history = clf_model.fit(train_gen, epochs=3, validation_data=val_gen, steps_per_epoch=num_iterations)
    
    # Calculate all relevant metrics
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])

  0%|          | 0/35 [00:00<?, ?it/s]

29097984/29084464 [==============================] - 0s 0us/step
Epoch 1/3
149/149 [==============================] - 101s 557ms/step - loss: 1.4606 - accuracy: 0.6343 - val_loss: 0.3214 - val_accuracy: 0.9200
Epoch 2/3
149/149 [==============================] - 72s 481ms/step - loss: 0.4372 - accuracy: 0.8943 - val_loss: 0.1158 - val_accuracy: 0.9733
Epoch 3/3
149/149 [==============================] - 71s 474ms/step - loss: 0.2785 - accuracy: 0.9265 - val_loss: 0.0780 - val_accuracy: 0.9867


  3%|▎         | 1/35 [05:22<3:03:01, 322.97s/it]

51888128/51877672 [==============================] - 0s 0us/step
Epoch 1/3
149/149 [==============================] - 92s 537ms/step - loss: 1.1717 - accuracy: 0.7255 - val_loss: 0.1831 - val_accuracy: 0.9467
Epoch 2/3
149/149 [==============================] - 74s 495ms/step - loss: 0.3141 - accuracy: 0.9219 - val_loss: 0.0648 - val_accuracy: 0.9933
Epoch 3/3
149/149 [==============================] - 73s 490ms/step - loss: 0.2062 - accuracy: 0.9462 - val_loss: 0.0436 - val_accuracy: 0.9800


  6%|▌         | 2/35 [09:39<2:36:06, 283.82s/it]

...

 94%|█████████▍| 33/35 [2:54:34<10:00, 300.01s/it]

80150528/80134624 [==============================] - 2s 0us/step
Epoch 1/3
149/149 [==============================] - 80s 527ms/step - loss: 3.0598 - accuracy: 0.2287 - val_loss: 2.5513 - val_accuracy: 0.5333
Epoch 2/3
149/149 [==============================] - 79s 525ms/step - loss: 2.4749 - accuracy: 0.4808 - val_loss: 1.9361 - val_accuracy: 0.6533
Epoch 3/3
149/149 [==============================] - 78s 525ms/step - loss: 2.0732 - accuracy: 0.5911 - val_loss: 1.5238 - val_accuracy: 0.7600


 97%|█████████▋| 34/35 [2:58:40<04:43, 283.92s/it]

83697664/83683744 [==============================] - 1s 0us/step
Epoch 1/3
149/149 [==============================] - 86s 535ms/step - loss: 0.8943 - accuracy: 0.7952 - val_loss: 0.1681 - val_accuracy: 0.9533
Epoch 2/3
149/149 [==============================] - 77s 513ms/step - loss: 0.3088 - accuracy: 0.9099 - val_loss: 0.1217 - val_accuracy: 0.9667
Epoch 3/3
149/149 [==============================] - 76s 511ms/step - loss: 0.2388 - accuracy: 0.9292 - val_loss: 0.0966 - val_accuracy: 0.9533


100%|██████████| 35/35 [3:02:49<00:00, 313.42s/it]


In [24]:
benchmark_df = pd.DataFrame(model_benchmarks)

# sort in ascending order of num_model_params column
benchmark_df.sort_values('validation_accuracy', inplace=True)

# write results to csv file
benchmark_df.to_csv('benchmark_df.csv', index=False)
benchmark_df

,model_name,num_model_params,validation_accuracy
11,EfficientNetV2B0,5919312,0.033333
10,EfficientNetB7,64097687,0.033333
3,EfficientNetB0,4049571,0.033333
4,EfficientNetB1,6575239,0.033333
5,EfficientNetB2,7768569,0.033333
6,EfficientNetB3,10783535,0.033333
7,EfficientNetB4,17673823,0.033333
8,EfficientNetB5,28513527,0.033333
9,EfficientNetB6,40960143,0.033333
16,EfficientNetV2M,53150388,0.040000


In [30]:
benchmark_df[benchmark_df["validation_accuracy"]>0.8]

,model_name,num_model_params,validation_accuracy
25,NASNetMobile,4269716,0.940000
24,NASNetLarge,84916818,0.940000
34,Xception,20861480,0.953333
18,InceptionResNetV2,54336736,0.953333
19,InceptionV3,21802784,0.966667
31,ResNet50V2,23564800,0.973333
27,ResNet101V2,42626560,0.980000
29,ResNet152V2,58331648,0.980000
1,DenseNet169,12642880,0.980000
21,MobileNetV2,2257984,0.980000
